In [8]:
import numpy as np
import cv2
import os
import pandas as pd
import tensorflow as tf
import random
import copy
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from google.colab import drive

drive.mount('/content/drive/')

## Folder path for the UTK image dataset

folder_path = '/content/drive/MyDrive/UTKFace/'

## Paths for the npy files created for ease

#images_path = '/content/drive/MyDrive/UTK/images.npy'

#ages_path = '/content/drive/MyDrive/data/ages.npy'

#genders_path = '/content/drive/MyDrive/data/genders.npy'

#races_path = '/content/drive/MyDrive/data/races.npy'

def load_images_from_folder(folder):
    images = []
    ages = []
    genders = []
    races = []
    dates = []
    for filename in os.listdir(folder):
        print(filename)
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
          if (len(filename.split('_')) == 4):
            age, gender, race, date = filename_data(filename)
            images.append(img)
            ages.append(age)
            genders.append(gender)
            races.append(race)
            dates.append(date)
    return images, ages, genders, races, dates


def filename_data(filename):
  img_name = filename
  data = img_name.split('_')
  age = data[0]
  gender = data[1]
  race = data[2]
  data2 = data[3]
  date = data2.split('.')[0]
  return age, gender, race, date

Mounted at /content/drive/


In [9]:
## This is just to save the data for later so loading isn't required every time
images, ages, genders, races, dates = load_images_from_folder(folder_path)
np.save('images.npy', images)
np.save('ages.npy', ages)
np.save('genders.npy', genders)
np.save('races.npy', races)
np.save('dates.npy', dates)

Streaming output truncated to the last 5000 lines.
10_0_0_20170110225505288.jpg.chip.jpg
10_0_0_20170110225502403.jpg.chip.jpg
10_0_0_20170110225518663.jpg.chip.jpg
10_0_0_20170110225528484.jpg.chip.jpg
10_0_0_20170110225557604.jpg.chip.jpg
10_0_0_20170110225546130.jpg.chip.jpg
10_0_0_20170110225537307.jpg.chip.jpg
10_0_1_20170110220507258.jpg.chip.jpg
10_0_1_20170110220523360.jpg.chip.jpg
10_0_0_20170110225601897.jpg.chip.jpg
10_0_1_20170110223455893.jpg.chip.jpg
10_0_1_20170110223848885.jpg.chip.jpg
10_0_1_20170110225121326.jpg.chip.jpg
10_0_2_20170110224230094.jpg.chip.jpg
10_1_1_20170110225155603.jpg.chip.jpg
10_1_0_20170110220649364.jpg.chip.jpg
10_0_1_20170110225339066.jpg.chip.jpg
11_0_0_20170110220518578.jpg.chip.jpg
11_0_0_20170110220500946.jpg.chip.jpg
11_0_0_20170110220408722.jpg.chip.jpg
11_0_0_20170110220453002.jpg.chip.jpg
11_0_0_20170110220710576.jpg.chip.jpg
11_0_0_20170110224233654.jpg.chip.jpg
11_0_0_20170110220657089.jpg.chip.jpg
11_0_0_20170110224340941.jpg.chip.jpg

In [10]:
## Paths for the npy files created for ease

images_path = 'images.npy'

ages_path = 'ages.npy'

genders_path = 'genders.npy'

races_path = 'races.npy'
## Loading
images = np.load(images_path)
ages = np.load(ages_path)
genders = np.load(genders_path)
races = np.load(races_path)

In [11]:
## gender: 0=male, 1=female
## race: 0=white, 1=black, 2=asian, 3=indian

genders = genders.astype(np.uint8)
images.shape
df = pd.DataFrame({
    'Age': ages,
    'Gender': genders,
    'Race': races
})
white = df.Race == '0'
black = df.Race == '1'
asian = df.Race == '2'
indian = df.Race == '3'

white_images = images[white]
white_genders = genders[white]
black_images = images[black]
black_genders = genders[black]
asian_images = images[asian]
asian_genders = genders[asian]
indian_images = images[indian]
indian_genders = genders[indian]

n_white = len(white_genders)
n_black = len(black_genders)
n_asian = len(asian_genders)
n_indian = len(indian_genders)
n = min(n_white,n_black,n_asian,n_indian)
random.seed(100)
white_idx = random.sample(range(n_white), n)
asian_idx = random.sample(range(n_asian), n)
indian_idx = random.sample(range(n_indian), n)

white_images_2 = white_images[white_idx]
white_genders_2 = white_genders[white_idx]
asian_images_2 = asian_images[asian_idx]
asain_genders_2 = asian_genders[asian_idx]
indian_images_2 = indian_images[indian_idx]
indian_genders_2 = indian_genders[indian_idx]

**MODEL**

In [12]:
## This is the model I made(also I'm not even sure if it's right but I tried)
def createModel():
  model = tf.keras.models.Sequential()

  model.add(
    layers.Conv2D(64, (4,4),
                  activation = 'relu',
                  input_shape = (200,200,3))
  )
  model.add(
    layers.MaxPooling2D((2,2))
  )
  model.add(
    layers.Conv2D(32, (2,2),
                  activation = 'relu')
  )
  model.add(
    layers.MaxPooling2D((2,2))
  )
  model.add(
    layers.Flatten()
  )
  model.add(
    layers.Dense(64, activation = 'relu')
  )
  model.add(
      layers.Dropout(.5)
  )
  model.add(
      layers.Dense(10, activation = 'relu')
  )
  model.add(
      layers.Dropout(.5)
  )
## Sigmoid for binary classification
  model.add(
    layers.Dense(1, activation = 'sigmoid')
  )
## Might need to change learning rate later
  model.compile(
  optimizer = optimizers.SGD(learning_rate=1e-9),
  loss = 'binary_crossentropy',
  metrics = ['accuracy']
  )
  return model

In [13]:
model = createModel()


In [14]:
temp_images = np.array(images[0:1000])
temp_genders = np.array(genders[0:1000])
model.fit(temp_images, temp_genders, epochs = 5)

Epoch 1/5
32/32 [==============================] - 34s 39ms/step - loss: 16.2525 - accuracy: 0.3994
Epoch 2/5
32/32 [==============================] - 1s 34ms/step - loss: 16.4496 - accuracy: 0.4166
Epoch 3/5
32/32 [==============================] - 1s 34ms/step - loss: 15.1119 - accuracy: 0.4468
Epoch 4/5
32/32 [==============================] - 1s 34ms/step - loss: 16.1986 - accuracy: 0.4268
Epoch 5/5
32/32 [==============================] - 1s 34ms/step - loss: 15.4827 - accuracy: 0.4211


In [15]:
from sklearn.metrics import classification_report

y_pred = model.predict(temp_images, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(temp_genders, y_pred_bool))

16/16 [==============================] - 1s 32ms/step
              precision    recall  f1-score   support

           0       0.70      1.00      0.82       698
           1       0.00      0.00      0.00       302

    accuracy                           0.70      1000
   macro avg       0.35      0.50      0.41      1000
weighted avg       0.49      0.70      0.57      1000



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [5]:
def createModel2():
  model2 = tf.keras.models.Sequential()

  model2.add(
    layers.Conv2D(128, (8,8),
                  activation = 'relu',
                  input_shape = (200,200,3))
  )
  model2.add(
    layers.MaxPooling2D((2,2))
  )
  model2.add(
    layers.Conv2D(64, (4,4),
                  activation = 'relu')
  )
  model2.add(
    layers.MaxPooling2D((2,2))
  )
  model2.add(
    layers.Conv2D(32, (2,2),
                  activation = 'relu')
  )
  model2.add(
    layers.MaxPooling2D((2,2))
  )
  model2.add(
    layers.Flatten()
  )
  model2.add(
    layers.Dense(128, activation = 'relu')
  )
  model2.add(
      layers.Dropout(.2)
  )
  model2.add(
    layers.Dense(64, activation = 'relu')
  )
  model2.add(
      layers.Dropout(.2)
  )
  model2.add(
      layers.Dense(10, activation = 'relu')
  )
  model2.add(
      layers.Dropout(.2)
  )
## Sigmoid for binary classification
  model2.add(
    layers.Dense(1, activation = 'sigmoid')
  )
## Might need to change learning rate later
  model2.compile(
  optimizer = optimizers.SGD(learning_rate=1e-9),
  loss = 'binary_crossentropy',
  metrics = ['accuracy']
  )
  return model2

In [6]:
model2 = createModel2()

In [7]:
from sklearn.model_selection import train_test_split

image_train,image_test,gender_train,gender_test = train_test_split(images,genders,test_size = 0.2)


NameError: ignored

In [1]:
model2.fit(image_train, gender_train, epochs = 5)

NameError: ignored